In [1]:
import chromadb

client = chromadb.PersistentClient(path="/chormadb/db")

client.heartbeat() # returns a nanosecond heartbeat. Useful for making sure the client remains connected.
# client.reset() # Empties and completely resets the database. ⚠️ This is destructive and not reversible.

1734122958076865900

In [2]:
client = chromadb.HttpClient(host='localhost', port=8000)
client.delete_collection(name="eu_sei_la")

InvalidArgumentError: Collection eu_sei_la does not exist.

In [3]:
import chromadb, json
from tqdm import tqdm
from llama_index.embeddings.gemini import GeminiEmbedding
import chromadb.utils.embedding_functions as embedding_functions
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)

# collection = client.create_collection(name="eu_sei_la")
api_key = open('../auth/gemini_key.txt').read()

# model_name = "models/embedding-001"
# embed_model = GeminiEmbedding(
#     model_name=model_name, api_key=api_key, title="this is a document"
# )
embed_model  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=api_key)

collection = client.get_or_create_collection(name="eu_sei_la", embedding_function=embed_model)

parsed_folder = '../BData/parsed_data/'
# print(collection)
js = json.loads(open(f'{parsed_folder}corpus.json', encoding="utf-8").read())

# for idx, it in js:
for idx, it in tqdm(enumerate(js), total=len(js)):
    txt, emb, doc, page = it['txt'], it['emb'], it['doc'], it['page']
    txt = txt.replace("'", '')
    # print(page, emb)
    collection.add(
        documents=[txt],
        # embeddings=[emb],
        metadatas=[{"doc": doc, "page": page}],
        ids=[f"document{doc}_page{page}_{idx}"]
    )

C:\Users\Henry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Henry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 725/725 [04:19<00:00,  2.80it/s]


In [4]:
print(collection.get(ids=["document2_page590_724"], include=['embeddings', 'metadatas']))

{'ids': ['document2_page590_724'], 'embeddings': array([[ 1.54530313e-02, -5.05000837e-02, -5.23613170e-02,
         1.82127953e-03,  4.17679027e-02,  1.84191391e-02,
         4.97428374e-03, -3.65496129e-02,  4.66005206e-02,
         6.69689998e-02,  4.04651128e-02,  1.06054200e-02,
         1.68936793e-02, -5.54256316e-04, -1.97796226e-02,
        -5.46865389e-02,  2.45543681e-02, -1.38266422e-02,
        -2.91402754e-03, -3.04790828e-02,  3.10290791e-03,
         2.25675814e-02,  1.24103623e-02, -3.95433903e-02,
        -5.46384044e-02,  5.28529054e-03,  1.87075082e-02,
        -1.01552024e-01,  1.49165746e-02, -3.40784118e-02,
        -4.99214567e-02,  3.88769433e-02, -6.66892380e-02,
        -1.66370291e-02, -2.68257633e-02, -1.14658829e-02,
         9.82141122e-03,  4.13455255e-02, -4.58106957e-02,
         2.21489314e-02, -2.36507021e-02, -2.82322727e-02,
        -9.09482688e-03, -4.67227362e-02,  1.86612122e-02,
        -3.56485024e-02,  9.72473901e-03,  2.28951611e-02,
       

In [10]:
query = "What is the maximum rating for RCBO in the small power system?"

results = collection.query(
    query_texts=[query],
    n_results=3
)
print(f"Query: {query}\n")
for idx, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"Document: {doc}")
    print(f"Distance: {distance}\n")

Query: What is the maximum rating for RCBO in the small power system?

Document: BUILDING USER GUIDE – TECHNICAL USER 
Royal United Hospitals, Dyson Cancer Centre, Bath 
© 2023 Riddell Project Management – Rev 2 (09.02.24) 
2.5 Small Power   
 
The small power systems generally comprise the assortment of socket outlets, isolating 
switches and fused connection units etc. The system is generally wired in LSOH single core 
cables, laid in trunkin g and/or drawn through conduits or wired in multi core copper 
XLPE/SWA/LS0H armoured cables, on cable tray / ladder.  
 
A network of cleaner’s sockets, generally installed at 9m intervals in corridors and within 
each occupied room, are installed around the hospital. Circuits for cleaner’s supplies are a 
ring arrangement on a dedicated cleaner’s circuit to min imise the risk of disruption to core 
hospital activities.  
 
Final circuits incorporating socket ou tlets are generally supplied via single pole combine d 
Residual curr ent devi ces 